In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('gon20-21data.txt')
winter2 = pd.read_csv ('gon21-22data.txt')
winter3 = pd.read_csv ('gon22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
gon = LocationInfo(
    'Groton', 'Connecticut', 
    latitude=41.3482,  
    longitude=-72.0787,
    timezone='America/Connecticut'
)

gon

LocationInfo(name='Groton', region='Connecticut', timezone='America/Connecticut', latitude=41.3482, longitude=-72.0787)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(gon.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(gon.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(gon.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:55:36  2020-12-01 16:19:19  2020-12-01
1   2020-12-02 06:56:38  2020-12-02 16:19:06  2020-12-02
2   2020-12-03 06:57:38  2020-12-03 16:18:54  2020-12-03
3   2020-12-04 06:58:37  2020-12-04 16:18:45  2020-12-04
4   2020-12-05 06:59:34  2020-12-05 16:18:38  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:30:15  2021-02-24 17:33:14  2021-02-24
86  2021-02-25 06:28:45  2021-02-25 17:34:26  2021-02-25
87  2021-02-26 06:27:14  2021-02-26 17:35:37  2021-02-26
88  2021-02-27 06:25:42  2021-02-27 17:36:47  2021-02-27
89  2021-02-28 06:24:09  2021-02-28 17:37:58  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:55:21  2021-12-01 16:19:23  2021-12-01
1   2021-12-02 06:56:23  2021-12-02 16:19:08  2021-12-02
2   2021-12-03 06:57:23  2021-12-03 16:18:56  2021-12-03
3   2021-12-04 06:58:23  2021-12-04 16:18:46  2021-12-04
4   2021-12-05 06:59:21  2021-12-05 16:18:39  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:30:37  2022-02-24 17:32:57  2022-02-24
86  2022-02-25 06:29:07  2022-02-25 17:34:08  2022-02-25
87  2022-02-26 06:27:36  2022-02-26 17:35:19  2022-02-26
88  2022-02-27 06:26:04  2022-02-27 17:36:30  2022-02-27
89  2022-02-28 06:24:32  2022-02-28 17:37:41  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:55:06  2022-12-01 16:19:26  2022-12-01
1   2022-12-02 06:56:08  2022-12-02 16:19:11  2022-12-02
2   2022-12-03 06:57:09  2022-12-03 16:18:59  2022-12-03
3   2022-12-04 06:58:08  2022-12-04 16:18:48  2022-12-04
4   2022-12-05 06:59:07  2022-12-05 16:18:40  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:30:58  2023-02-24 17:32:40  2023-02-24
86  2023-02-25 06:29:28  2023-02-25 17:33:51  2023-02-25
87  2023-02-26 06:27:58  2023-02-26 17:35:02  2023-02-26
88  2023-02-27 06:26:26  2023-02-27 17:36:13  2023-02-27
89  2023-02-28 06:24:54  2023-02-28 17:37:24  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GON,2020-12-01 00:56:00,0.7600,-RA,2020-12-01,2020-12-01 06:55:36,2020-12-01 16:19:19
1,GON,2020-12-02 17:56:00,0.0001,-RA,2020-12-02,2020-12-02 06:56:38,2020-12-02 16:19:06
2,GON,2020-12-04 20:56:00,0.0001,-RA,2020-12-04,2020-12-04 06:58:37,2020-12-04 16:18:45
3,GON,2020-12-04 21:56:00,0.2500,-RA,2020-12-04,2020-12-04 06:58:37,2020-12-04 16:18:45
4,GON,2020-12-04 22:56:00,0.5100,-RA,2020-12-04,2020-12-04 06:58:37,2020-12-04 16:18:45
...,...,...,...,...,...,...,...
205,GON,2021-02-27 15:56:00,1.0200,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47
206,GON,2021-02-27 16:56:00,1.5200,RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47
207,GON,2021-02-27 17:56:00,1.7800,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47
208,GON,2021-02-27 18:56:00,0.7600,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GON,2021-12-02 07:56:00,0.0001,-RA,2021-12-02,2021-12-02 06:56:23,2021-12-02 16:19:08
1,GON,2021-12-03 04:56:00,0.2500,-RA,2021-12-03,2021-12-03 06:57:23,2021-12-03 16:18:56
2,GON,2021-12-06 09:56:00,3.5600,RA BR,2021-12-06,2021-12-06 07:00:17,2021-12-06 16:18:34
3,GON,2021-12-07 01:56:00,0.5100,-RA,2021-12-07,2021-12-07 07:01:13,2021-12-07 16:18:31
4,GON,2021-12-07 02:56:00,0.5100,RA BR,2021-12-07,2021-12-07 07:01:13,2021-12-07 16:18:31
...,...,...,...,...,...,...,...
241,GON,2022-02-25 14:56:00,1.7800,FZRA BR,2022-02-25,2022-02-25 06:29:07,2022-02-25 17:34:08
242,GON,2022-02-25 15:56:00,2.2900,-FZRA,2022-02-25,2022-02-25 06:29:07,2022-02-25 17:34:08
243,GON,2022-02-25 17:56:00,0.0001,-RA BR,2022-02-25,2022-02-25 06:29:07,2022-02-25 17:34:08
244,GON,2022-02-25 18:56:00,0.0001,-RA BR,2022-02-25,2022-02-25 06:29:07,2022-02-25 17:34:08


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GON,2022-12-01 00:56:00,0.2500,-RA,2022-12-01,2022-12-01 06:55:06,2022-12-01 16:19:26
1,GON,2022-12-03 14:56:00,0.0001,-RA,2022-12-03,2022-12-03 06:57:09,2022-12-03 16:18:59
2,GON,2022-12-03 15:56:00,0.5100,-RA,2022-12-03,2022-12-03 06:57:09,2022-12-03 16:18:59
3,GON,2022-12-03 16:56:00,0.2500,-RA,2022-12-03,2022-12-03 06:57:09,2022-12-03 16:18:59
4,GON,2022-12-03 17:56:00,2.5400,RA,2022-12-03,2022-12-03 06:57:09,2022-12-03 16:18:59
...,...,...,...,...,...,...,...
233,GON,2023-02-25 16:56:00,0.0001,-SN,2023-02-25,2023-02-25 06:29:28,2023-02-25 17:33:51
234,GON,2023-02-25 17:56:00,0.0001,-SN,2023-02-25,2023-02-25 06:29:28,2023-02-25 17:33:51
235,GON,2023-02-25 18:56:00,0.5100,-SN,2023-02-25,2023-02-25 06:29:28,2023-02-25 17:33:51
236,GON,2023-02-25 19:56:00,0.0001,-SN,2023-02-25,2023-02-25 06:29:28,2023-02-25 17:33:51


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
10,GON,2020-12-05 07:56:00,0.5100,-RA BR,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38
11,GON,2020-12-05 08:56:00,0.5100,-RA,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38
12,GON,2020-12-05 09:56:00,2.5400,RA BR,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38
13,GON,2020-12-05 10:56:00,3.5600,RA BR,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38
14,GON,2020-12-05 11:56:00,1.5200,-RA,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38
...,...,...,...,...,...,...,...
202,GON,2021-02-27 12:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47
203,GON,2021-02-27 13:56:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47
204,GON,2021-02-27 14:56:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47
205,GON,2021-02-27 15:56:00,1.0200,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
11,GON,2020-12-05 08:56:00,0.5100,-RA,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38,2020-12-05 07:59:34,2020-12-05 15:18:38
12,GON,2020-12-05 09:56:00,2.5400,RA BR,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38,2020-12-05 07:59:34,2020-12-05 15:18:38
13,GON,2020-12-05 10:56:00,3.5600,RA BR,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38,2020-12-05 07:59:34,2020-12-05 15:18:38
14,GON,2020-12-05 11:56:00,1.5200,-RA,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38,2020-12-05 07:59:34,2020-12-05 15:18:38
15,GON,2020-12-05 12:56:00,3.3000,RA BR,2020-12-05,2020-12-05 06:59:34,2020-12-05 16:18:38,2020-12-05 07:59:34,2020-12-05 15:18:38
...,...,...,...,...,...,...,...,...,...
201,GON,2021-02-27 11:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47,2021-02-27 07:25:42,2021-02-27 16:36:47
202,GON,2021-02-27 12:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47,2021-02-27 07:25:42,2021-02-27 16:36:47
203,GON,2021-02-27 13:56:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47,2021-02-27 07:25:42,2021-02-27 16:36:47
204,GON,2021-02-27 14:56:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47,2021-02-27 07:25:42,2021-02-27 16:36:47


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.449473611111111
2021-2022: 0.8717156626506024
2022-2023: 0.8048015384615382


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GON,2020-12-01 00:56:00,0.7600,-RA,2020-12-01,2020-12-01 06:55:36,2020-12-01 16:19:19
1,GON,2020-12-02 17:56:00,0.0001,-RA,2020-12-02,2020-12-02 06:56:38,2020-12-02 16:19:06
2,GON,2020-12-04 20:56:00,0.0001,-RA,2020-12-04,2020-12-04 06:58:37,2020-12-04 16:18:45
3,GON,2020-12-04 21:56:00,0.2500,-RA,2020-12-04,2020-12-04 06:58:37,2020-12-04 16:18:45
4,GON,2020-12-04 22:56:00,0.5100,-RA,2020-12-04,2020-12-04 06:58:37,2020-12-04 16:18:45
...,...,...,...,...,...,...,...
197,GON,2021-02-23 19:56:00,0.0001,-RA BR,2021-02-23,2021-02-23 06:31:44,2021-02-23 17:32:03
198,GON,2021-02-23 20:56:00,0.0001,-RA BR,2021-02-23,2021-02-23 06:31:44,2021-02-23 17:32:03
207,GON,2021-02-27 17:56:00,1.7800,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47
208,GON,2021-02-27 18:56:00,0.7600,-RA BR,2021-02-27,2021-02-27 06:25:42,2021-02-27 17:36:47


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20628\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.9978885714285716
2021-2022: 0.39065200000000005
2022-2023: 0.7792216216216216
